---

## Get FIRMS API Key (MAP_KEY)

---

In order to provide access to FIRMS data, we require our users to sign up for a **FREE** API / map key (we call it: MAP_KEY).

The MAP_KEY was designed to conserve FIRMS resources, so everyone could have a reasonable access to our data. For example, a download script with an error could end up requesting too much data or query our database with high frequency.

FIRMS MAP_KEY was originally designed to facilitate only mapserver visualization queries, but now can be used for other requests. For example:

- Web Map Service (WMS)
- Web Feature Service (WFS)
- API

To sign up, visit https://firms.modaps.eosdis.nasa.gov/api/map_key

FIRMS MAP_KEY limits your usage to a 10-minute window. So if you exceed your limit, after 10 minutes, it resets so you can use the system again.

---

## Test Your MAP_KEY

---

In [1]:
# Let's set your map key that was emailed to you. It should look something like 'abcdef1234567890abcdef1234567890'
MAP_KEY = '5e8bad8d50fa1ca84ea72175e2bace34' #key from guy.80647@gmail.com
#MAP_KEY = 'abcdef1234567890abcdef1234567890'

# now let's check how many transactions we have
import pandas as pd
url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
try:
  df = pd.read_json(url,  typ='series')
  display(df)
except:
  # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
  print ("There is an issue with the query. \nTry in your browser: %s" % url)


transaction_limit             5000
current_transactions           108
transaction_interval    10 minutes
dtype: object

In [2]:
# let's create a simple function that tells us how many transactions we have used.
# We will use this in later examples

def get_transaction_count() :
  count = 0
  try:
    df = pd.read_json(url,  typ='series')
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('Our current transaction count is %i' % tcount)

Our current transaction count is 0


---

## API/data_availability

---

This service is designed to inform users about date range availability of our supported datasets.

For more information visit https://firms.modaps.eosdis.nasa.gov/api/data_availability

Let's see the full list of available sensors and their supported date ranges.


In [2]:
# let's query data_availability to find out what date range is available for various datasets
# we will explain these datasets a bit later

# this url will return information about all supported sensors and their corresponding datasets
# instead of 'all' you can specify individual sensor, ex:LANDSAT_NRT
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pd.read_csv(da_url)
display(df)

,data_id,min_date,max_date
0,MODIS_NRT,2025-01-01,2025-04-17
1,MODIS_SP,2000-11-01,2024-12-31
2,VIIRS_NOAA20_NRT,2025-02-01,2025-04-17
3,VIIRS_NOAA20_SP,2018-04-01,2025-01-31
4,VIIRS_NOAA21_NRT,2024-01-17,2025-04-17
5,VIIRS_SNPP_NRT,2025-02-01,2025-04-17
6,VIIRS_SNPP_SP,2012-01-20,2025-01-31
7,LANDSAT_NRT,2022-06-20,2025-04-16
8,GOES_NRT,2022-08-09,2025-04-17
9,BA_MODIS,2000-11-01,2024-12-01


**data_id** column shows the dataset id which we will need in later queries:
- 'NRT' means this is Near Real-Time dataset but it may also includes Real Time (RT) and Ultra Real Time (URT) data [click here more info on URT/RT](https://www.earthdata.nasa.gov/data/tools/firms/faq)
- 'SP' or Standard Processing; standard data products are an internally consistent, well-calibrated record of the Earth’s geophysical properties to support science. There is a multi-month lag in this dataset availability. [more information on SP vs NRT](https://www.earthdata.nasa.gov/data/tools/firms/faq)
- BA_MODIS is for MODIS burned areas product

**min_date** and **max_date** columns provide the available date range for these datasets. Dates are based on GMT

In [3]:
# now let's see how many transactions we use by querying this end point

start_count = get_transaction_count()
pd.read_csv(da_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

# now remember, after 10 minutes this will reset

NameError: name 'get_transaction_count' is not defined

---

## API/area

---

Fire detection hotspots based on area, date and sensor. For more information visit https://firms.modaps.eosdis.nasa.gov/api/area

The end point expects these parameters: [MAP_KEY], [SOURCE], [AREA_COORDINATES],[DAY_RANGE] and optionally [DATE] for historical data

**NOTE** - querying the entire world for VIIRS can return between 30,000 - 100,000+ records per day


In [4]:
# in this example let's look at VIIRS NOAA-20, entire world and the most recent day
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/world/1'
start_count = get_transaction_count()
df_area = pd.read_csv(area_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

df_area

NameError: name 'get_transaction_count' is not defined

In [2]:
# We can also focus on a smaller area ex. South Asia and get the last 3 days of records
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/54,5.5,102,40/6'
df_area = pd.read_csv(area_url)
df_area

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,6.98998,100.75453,333.82,0.61,0.71,2025-04-30,548,N20,VIIRS,n,2.0NRT,291.88,5.51,D
1,6.99030,100.75359,332.27,0.61,0.71,2025-04-30,548,N20,VIIRS,n,2.0NRT,292.34,4.80,D
2,11.75490,99.59682,330.85,0.60,0.71,2025-04-30,550,N20,VIIRS,n,2.0NRT,288.33,5.21,D
3,11.76555,99.51774,342.72,0.61,0.71,2025-04-30,550,N20,VIIRS,n,2.0NRT,289.88,7.04,D
4,13.34769,99.77871,332.39,0.55,0.68,2025-04-30,550,N20,VIIRS,n,2.0NRT,295.33,3.01,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18261,18.17535,55.24595,313.41,0.49,0.40,2025-05-04,2210,N20,VIIRS,n,2.0NRT,295.83,1.10,N
18262,18.17638,55.24849,322.39,0.49,0.41,2025-05-04,2210,N20,VIIRS,n,2.0NRT,297.29,1.96,N
18263,18.56183,55.86657,310.87,0.52,0.42,2025-05-04,2210,N20,VIIRS,n,2.0NRT,297.22,1.17,N
18264,18.64210,55.41883,329.63,0.50,0.41,2025-05-04,2210,N20,VIIRS,n,2.0NRT,296.79,5.73,N


List of supported countries and their 3-letter codes. This may be easier to read in html format https://firms.modaps.eosdis.nasa.gov/api/countries/?format=html however, you won't be able to see the exent box defined for each country.

Example below shows how you can view from Python.

In [5]:
# We can also focus on smaller area ex. South Asia and get last 3 days of records
countries_url = 'https://firms.modaps.eosdis.nasa.gov/api/countries'
df_countries = pd.read_csv(countries_url, sep=';')
df_countries

,id,abreviation,name,extent
0,1,ABW,Aruba,"BOX(-70.0624080069999 12.417669989,-69.8768204..."
1,2,AFG,Afghanistan,"BOX(60.4867777910001 29.3866053260001,74.89230..."
2,3,AGO,Angola,"BOX(11.6693941430001 -18.0314047239998,24.0617..."
3,4,AIA,Anguilla,"BOX(-63.4288223949999 18.1690941430001,-62.972..."
4,6,ALA,Aland Islands,"BOX(19.5131942070001 59.9044863950001,21.09669..."
...,...,...,...,...
239,234,WSM,Samoa,"BOX(-172.782582161 -14.052829685,-171.43769283..."
240,235,YEM,Yemen,"BOX(42.5457462900001 12.1114436720001,54.54029..."
241,236,ZAF,South Africa,"BOX(16.4699813160001 -46.965752863,37.97779381..."
242,237,ZMB,Zambia,"BOX(21.9798775630001 -18.0692318719999,33.6742..."


---

## API/country

---

Provides data specific to a country, although not recommended for large countries such as USA, China, Canada, Russia due to the complexity and size of their polygon shape which may cause the query to time out. To figure out the country code, see [/api/countries](https://firms.modaps.eosdis.nasa.gov/api/countries/?format=html) (example above).

In [3]:
# Let's see last four days MODIS data for Thailand
thai_url = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_NRT/THA/6'
df_thai = pd.read_csv(thai_url)
df_thai

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,THA,15.64000,104.76543,315.43,1.49,1.20,2025-04-30,219,Terra,MODIS,31,6.1NRT,286.70,8.88,D
1,THA,12.13595,99.87524,323.46,1.16,1.07,2025-04-30,803,Aqua,MODIS,44,6.1NRT,302.60,8.87,D
2,THA,14.34006,100.04013,323.38,1.26,1.11,2025-04-30,803,Aqua,MODIS,72,6.1NRT,303.60,12.92,D
3,THA,14.41119,100.49567,320.12,1.35,1.15,2025-04-30,803,Aqua,MODIS,49,6.1NRT,294.89,13.14,D
4,THA,14.41281,100.50782,326.60,1.35,1.15,2025-04-30,803,Aqua,MODIS,76,6.1NRT,295.36,22.91,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,THA,15.73356,100.82130,304.73,2.05,1.39,2025-05-04,1410,Terra,MODIS,61,6.1NRT,272.16,18.22,N
90,THA,15.73711,100.81437,300.97,2.05,1.39,2025-05-04,1410,Terra,MODIS,36,6.1NRT,272.10,13.07,N
91,THA,18.07452,103.96301,309.65,1.16,1.07,2025-05-04,1410,Terra,MODIS,77,6.1NRT,292.27,8.06,N
92,THA,18.07610,103.97359,316.76,1.15,1.07,2025-05-04,1410,Terra,MODIS,94,6.1NRT,292.40,14.99,N


## Change data format "acq_time"

In [4]:
import datetime

# Ensure "acq_time" is a string, then pad with zeros
df_thai["acq_time"] = df_thai["acq_time"].astype(str).str.zfill(4)

# Convert to HH:MM format
df_thai["acq_time"] = pd.to_datetime(df_thai["acq_time"], format="%H%M").dt.time
# df_thai["acq_time"] = df_thai["acq_time"].apply(lambda x: datetime.datetime.strptime(x, "%H:%M:%S").time())

# Check the result
print(df_thai.dtypes)
print(df_thai["acq_time"].head())  # Display first few converted times

country_id     object
latitude      float64
longitude     float64
brightness    float64
scan          float64
track         float64
acq_date       object
acq_time       object
satellite      object
instrument     object
confidence      int64
version        object
bright_t31    float64
frp           float64
daynight       object
dtype: object
0    02:19:00
1    08:03:00
2    08:03:00
3    08:03:00
4    08:03:00
Name: acq_time, dtype: object


In [5]:
df_thai['acq_datetime'] = pd.to_datetime(df_thai['acq_date'].astype(str) + ' ' + df_thai['acq_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
df_thai['acq_datetime'].head()

0   2025-04-30 02:19:00
1   2025-04-30 08:03:00
2   2025-04-30 08:03:00
3   2025-04-30 08:03:00
4   2025-04-30 08:03:00
Name: acq_datetime, dtype: datetime64[ns]

In [6]:
import pytz
#change timazone to Bangkok
df_thai['acq_datetime_th'] = df_thai['acq_datetime'].dt.tz_localize('GMT').dt.tz_convert('Asia/Bangkok')
df_thai['acq_datetime_th'].head(5)


0   2025-04-30 09:19:00+07:00
1   2025-04-30 15:03:00+07:00
2   2025-04-30 15:03:00+07:00
3   2025-04-30 15:03:00+07:00
4   2025-04-30 15:03:00+07:00
Name: acq_datetime_th, dtype: datetime64[ns, Asia/Bangkok]

In [8]:
unique_values = df_thai["acq_time"].unique()
print(unique_values)

[datetime.time(2, 19) datetime.time(8, 3) datetime.time(14, 44)
 datetime.time(2, 58) datetime.time(3, 0) datetime.time(7, 1)
 datetime.time(13, 48) datetime.time(7, 41) datetime.time(14, 27)
 datetime.time(14, 29) datetime.time(2, 38) datetime.time(8, 22)
 datetime.time(20, 29) datetime.time(3, 17) datetime.time(7, 20)
 datetime.time(7, 22) datetime.time(14, 10)]


In [11]:
df_thai.head(100)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,15.64000,104.76543,315.43,1.49,1.20,2025-04-30,02:19:00,Terra,MODIS,...,286.70,8.88,D,2025-04-30 02:19:00,2025-04-30 09:19:00+07:00,2025,4,30,9,19
1,THA,12.13595,99.87524,323.46,1.16,1.07,2025-04-30,08:03:00,Aqua,MODIS,...,302.60,8.87,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
2,THA,14.34006,100.04013,323.38,1.26,1.11,2025-04-30,08:03:00,Aqua,MODIS,...,303.60,12.92,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
3,THA,14.41119,100.49567,320.12,1.35,1.15,2025-04-30,08:03:00,Aqua,MODIS,...,294.89,13.14,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
4,THA,14.41281,100.50782,326.60,1.35,1.15,2025-04-30,08:03:00,Aqua,MODIS,...,295.36,22.91,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,THA,15.73356,100.82130,304.73,2.05,1.39,2025-05-04,14:10:00,Terra,MODIS,...,272.16,18.22,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
90,THA,15.73711,100.81437,300.97,2.05,1.39,2025-05-04,14:10:00,Terra,MODIS,...,272.10,13.07,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
91,THA,18.07452,103.96301,309.65,1.16,1.07,2025-05-04,14:10:00,Terra,MODIS,...,292.27,8.06,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
92,THA,18.07610,103.97359,316.76,1.15,1.07,2025-05-04,14:10:00,Terra,MODIS,...,292.40,14.99,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10


## Hive partition

In [7]:
import numpy as np
import pandas as pd
import pyarrow

In [12]:
# Extract components from the "acq_date" and "acq_time" columns
df_thai["acq_year"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.year
df_thai["acq_month"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.month
df_thai["acq_day"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.day
df_thai["acq_hour"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.hour
df_thai["acq_minute"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.minute


In [15]:
df_thai.head(5)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,15.64000,104.76543,315.43,1.49,1.20,2025-04-30,02:19:00,Terra,MODIS,...,286.70,8.88,D,2025-04-30 02:19:00,2025-04-30 09:19:00+07:00,2025,4,30,9,19
1,THA,12.13595,99.87524,323.46,1.16,1.07,2025-04-30,08:03:00,Aqua,MODIS,...,302.60,8.87,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
2,THA,14.34006,100.04013,323.38,1.26,1.11,2025-04-30,08:03:00,Aqua,MODIS,...,303.60,12.92,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
3,THA,14.41119,100.49567,320.12,1.35,1.15,2025-04-30,08:03:00,Aqua,MODIS,...,294.89,13.14,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
4,THA,14.41281,100.50782,326.60,1.35,1.15,2025-04-30,08:03:00,Aqua,MODIS,...,295.36,22.91,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3


In [13]:
df_thai.to_parquet(
    "firms_outout",
    partition_cols=["acq_year","acq_month","acq_day","acq_hour","acq_minute"],   # <-- crucial for partitioning by retrieval_time
    engine="pyarrow",
    index=False
)

In [15]:
df_read = pd.read_parquet("firms_outout", engine="pyarrow")
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype                       
---  ------           --------------  -----                       
 0   country_id       282 non-null    object                      
 1   latitude         282 non-null    float64                     
 2   longitude        282 non-null    float64                     
 3   brightness       282 non-null    float64                     
 4   scan             282 non-null    float64                     
 5   track            282 non-null    float64                     
 6   acq_date         282 non-null    object                      
 7   acq_time         282 non-null    object                      
 8   satellite        282 non-null    object                      
 9   instrument       282 non-null    object                      
 10  confidence       282 non-null    int64                       
 11  version          28

In [16]:
df_read

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,12.13595,99.87524,323.46,1.16,1.07,2025-04-30,08:03:00,Aqua,MODIS,...,302.60,8.87,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
1,THA,14.34006,100.04013,323.38,1.26,1.11,2025-04-30,08:03:00,Aqua,MODIS,...,303.60,12.92,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
2,THA,14.41119,100.49567,320.12,1.35,1.15,2025-04-30,08:03:00,Aqua,MODIS,...,294.89,13.14,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
3,THA,14.41281,100.50782,326.60,1.35,1.15,2025-04-30,08:03:00,Aqua,MODIS,...,295.36,22.91,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
4,THA,16.27660,99.75736,324.49,1.28,1.12,2025-04-30,08:03:00,Aqua,MODIS,...,301.52,10.83,D,2025-04-30 08:03:00,2025-04-30 15:03:00+07:00,2025,4,30,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,THA,18.07610,103.97359,316.76,1.15,1.07,2025-05-04,14:10:00,Terra,MODIS,...,292.40,14.99,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
278,THA,19.70258,98.09076,302.77,2.76,1.59,2025-05-04,14:10:00,Terra,MODIS,...,291.74,16.23,N,2025-05-04 14:10:00,2025-05-04 21:10:00+07:00,2025,5,4,21,10
279,THA,17.54023,99.73300,302.68,1.75,1.30,2025-05-03,20:29:00,Aqua,MODIS,...,290.33,7.54,N,2025-05-03 20:29:00,2025-05-04 03:29:00+07:00,2025,5,4,3,29
280,THA,17.54023,99.73300,302.68,1.75,1.30,2025-05-03,20:29:00,Aqua,MODIS,...,290.33,7.54,N,2025-05-03 20:29:00,2025-05-04 03:29:00+07:00,2025,5,4,3,29


In [ ]:
# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "firms.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

In [ ]:
# Write DataFrame to a directory "output_parquet" partitioned by retrieval_time
df_thai.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=["acq_year","acq_month","acq_day","acq_hour","acq_minute"],   # <-- crucial for partitioning by retrieval_time
)

In [ ]:
#read parquet
path_all_partition = 's3a://weather/main/firms.parquet'

df2=pd.read_parquet(    
    path=path_all_partition,
    storage_options=storage_options
)
df2.info()
df2.head()